In [1]:
import pandas as pd
import requests
import json
import datetime
import concurrent.futures
import swifter
import os
import numpy as np
import matplotlib.pyplot as plt
from numpy import random

Matplotlib created a temporary cache directory at /tmp/matplotlib-0v9lrdgz because the default path (/opt/spark/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

In [3]:
df_4jefp = pd.read_csv('dados/TJRN_4º_JUIZADO_ESPECIAL_DA_FAZENDA_PÚBLICA_2022-01-01.csv', header='infer', sep=';', compression='zip')
df_4jefp.drop_duplicates(inplace=True)
if 'sort' in df_4jefp.columns:
    df_4jefp.drop(columns=['sort'])
if 'grau' in df_4jefp.columns:
    df_4jefp.drop(columns=['grau'])
if 'orgao_julgador' in df_4jefp.columns:
    df_4jefp.drop(columns=['orgao_julgador'])

In [4]:
df_4jefp.head(5)

,classe,data_ajuizamento,ultima_atualizacao,codigo,orgao_julgador,municipio,grau,assuntos,movimentos,sort
0,Procedimento do Juizado Especial da Fazenda Pública,2022-03-18 06:52:19-03:00,2022-10-01 00:47:57.171000-03:00,75496,4º JUIZADO ESPECIAL DA FAZENDA PÚBLICA,3201,JE,"['', '']","[[26, 'Distribuição', Timestamp('2022-03-17 21:00:00-0300', tz='America/Sao_Paulo')], [51, 'Conclusão', Timestamp('2022-03-17 21:00:00-0300', tz='America/Sao_Paulo')], [60, 'Expedição de documento', Timestamp('2022-04-26 21:00:00-0300', tz='America/Sao_Paulo')], [36, 'Redistribuição', Timestamp('2022-04-26 21:00:00-0300', tz='America/Sao_Paulo')], [51, 'Conclusão', Timestamp('2022-05-05 21:00:00-0300', tz='America/Sao_Paulo')], [60, 'Expedição de documento', Timestamp('2022-05-19 21:00:00-0300', tz='America/Sao_Paulo')], [941, 'Incompetência', Timestamp('2022-03-21 21:00:00-0300', tz='America/Sao_Paulo')], [12099, 'Por decisão do Presidente do STJ - SIRDR', Timestamp('2022-05-18 21:00:00-0300', tz='America/Sao_Paulo')]]",1681390223922
1,Procedimento do Juizado Especial da Fazenda Pública,2022-04-29 07:13:44-03:00,2022-10-01 00:47:15.166000-03:00,75496,4º JUIZADO ESPECIAL DA FAZENDA PÚBLICA,3201,JE,[''],"[[26, 'Distribuição', Timestamp('2022-04-28 21:00:00-0300', tz='America/Sao_Paulo')], [51, 'Conclusão', Timestamp('2022-04-28 21:00:00-0300', tz='America/Sao_Paulo')], [60, 'Expedição de documento', Timestamp('2022-05-12 21:00:00-0300', tz='America/Sao_Paulo')], [60, 'Expedição de documento', Timestamp('2022-05-12 21:00:00-0300', tz='America/Sao_Paulo')], [85, 'Petição', Timestamp('2022-05-29 21:00:00-0300', tz='America/Sao_Paulo')], [85, 'Petição', Timestamp('2022-06-16 21:00:00-0300', tz='America/Sao_Paulo')], [51, 'Conclusão', Timestamp('2022-07-11 21:00:00-0300', tz='America/Sao_Paulo')], [60, 'Expedição de documento', Timestamp('2022-07-26 21:00:00-0300', tz='America/Sao_Paulo')], [11010, 'Mero expediente', Timestamp('2022-05-11 21:00:00-0300', tz='America/Sao_Paulo')], [246, 'Definitivo', Timestamp('2022-08-25 21:00:00-0300', tz='America/Sao_Paulo')], [221, 'Procedência em Parte', Timestamp('2022-07-25 21:00:00-0300', tz='America/Sao_Paulo')]]",1681390228754
2,Procedimento do Juizado Especial da Fazenda Pública,2022-03-11 09:11:38-03:00,2022-10-01 00:47:05.012000-03:00,75496,4º JUIZADO ESPECIAL DA FAZENDA PÚBLICA,3201,JE,[''],"[[26, 'Distribuição', Timestamp('2022-03-10 21:00:00-0300', tz='America/Sao_Paulo')], [51, 'Conclusão', Timestamp('2022-03-10 21:00:00-0300', tz='America/Sao_Paulo')], [60, 'Expedição de documento', Timestamp('2022-03-14 21:00:00-0300', tz='America/Sao_Paulo')], [85, 'Petição', Timestamp('2022-04-04 21:00:00-0300', tz='America/Sao_Paulo')], [60, 'Expedição de documento', Timestamp('2022-05-30 21:00:00-0300', tz='America/Sao_Paulo')], [60, 'Expedição de documento', Timestamp('2022-05-30 21:00:00-0300', tz='America/Sao_Paulo')], [85, 'Petição', Timestamp('2022-06-05 21:00:00-0300', tz='America/Sao_Paulo')], [85, 'Petição', Timestamp('2022-07-04 21:00:00-0300', tz='America/Sao_Paulo')], [51, 'Conclusão', Timestamp('2022-07-11 21:00:00-0300', tz='America/Sao_Paulo')], [60, 'Expedição de documento', Timestamp('2022-08-01 21:00:00-0300', tz='America/Sao_Paulo')], [85, 'Petição', Timestamp('2022-08-25 21:00:00-0300', tz='America/Sao_Paulo')], [85, 'Petição', Timestamp('2022-09-13 21:00:00-0300', tz='America/Sao_Paulo')], [51, 'Conclusão', Timestamp('2022-09-14 21:00:00-0300', tz='America/Sao_Paulo')], [785, 'Antecipação de tutela', Timestamp('2022-03-13 21:00:00-0300', tz='America/Sao_Paulo')], [246, 'Definitivo', Timestamp('2022-09-12 21:00:00-0300', tz='America/Sao_Paulo')], [221, 'Procedência em Parte', Timestamp('2022-07-31 21:00:00-0300', tz='America/Sao_Paulo')]]",1681390228758
3,Procedimento do Juizado Especial da Fazenda Pública,2022-02-08 10:36:00-03:00,2022-10-01 00:47:02.716000-03:00,75496,4º JUIZADO ESPECIAL DA FAZENDA PÚBLICA,3201,JE,[''],"[[26, 'Distribuição', Timestamp('2022-02-07 21:00:00-0300', tz='America/Sao_

In [5]:
print(df_4jefp.info())
df_4jefp['data_ajuizamento'] = pd.to_datetime(df_4jefp['data_ajuizamento']).dt.strftime('%Y-%m-%d')
df_4jefp['ultima_atualizacao'] = pd.to_datetime(df_4jefp['ultima_atualizacao'], format='mixed').dt.strftime('%Y-%m-%d')
print(df_4jefp.head(2))

<class 'pandas.core.frame.DataFrame'>
Index: 11386 entries, 0 to 1383683
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   classe              11386 non-null  object
 1   data_ajuizamento    11386 non-null  object
 2   ultima_atualizacao  11386 non-null  object
 3   codigo              11386 non-null  int64 
 4   orgao_julgador      11386 non-null  object
 5   municipio           11386 non-null  int64 
 6   grau                11386 non-null  object
 7   assuntos            11386 non-null  object
 8   movimentos          11386 non-null  object
 9   sort                11386 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 978.5+ KB
None
                                                classe data_ajuizamento  \
0  Procedimento do Juizado Especial da Fazenda Pública       2022-03-18   
1  Procedimento do Juizado Especial da Fazenda Pública       2022-04-29   

  ultima_atualizacao  codigo       

In [6]:
for movimento in df_4jefp['movimentos']:
    #print(movimento)
    if (len(list(df_4jefp['movimentos'][10].split(','))) / 4) < 5:
        print(movimento)
        
print(len(list(df_4jefp['movimentos'][10].split(','))) / 4)
print(df_4jefp['movimentos'][10].split(',').index)

13.0
<built-in method index of list object at 0x7f754f62de80>


In [7]:
movimentacoes = list(df_4jefp.movimentos.values)

In [8]:
for mov in movimentacoes[5000:5010]:
    print(mov)
    
print('')
lista_movimentacoes_completa = list()
lista_codigos = list()
lista_nomes = list()
lista_data = list()
i = 0

for movimento in movimentacoes:
    if '272' in movimento.split():
        print(movimento)
        break 
        #print(len(list(movimento.split(',')))/4)
    else:
        if len(list(movimento.split(','))) % 4 == 0:
            for elemento in range(0,int(len(list(movimento.split(',')))/4),4):
                if elemento != None:
                    codigo = movimento.split(',')[elemento].replace('[','').replace(' ', '')
                    nome_movimentacao = movimento.split(',')[elemento+1].replace('[','').replace(' ', '')
                    data_movimentacao = movimento.split(',')[elemento+2].replace('[','').replace('Timestamp(','').split(' ')[1].replace(' ', '').replace('\'', '')
                    lista_temp = list()
                    lista_temp.append(codigo)
                    lista_temp.append(nome_movimentacao)
                    lista_temp.append(data_movimentacao)
                    lista_movimentacoes_completa.append([])
                    lista_movimentacoes_completa[i].append(lista_temp)
                else:
                    lista_temp = [-1, -1, -1]
                    lista_movimentacoes_completa.append([])
                    lista_movimentacoes_completa[i].append(lista_temp)
        i = i + 1

[[26, 'Distribuição', Timestamp('2023-08-14 08:20:38-0300', tz='America/Sao_Paulo')], [51, 'Conclusão', Timestamp('2023-08-14 08:20:41-0300', tz='America/Sao_Paulo')], [60, 'Expedição de documento', Timestamp('2023-09-12 10:44:56-0300', tz='America/Sao_Paulo')], [11010, 'Mero expediente', Timestamp('2023-09-04 11:58:56-0300', tz='America/Sao_Paulo')]]
[[51, 'Conclusão', Timestamp('2023-08-04 08:15:52-0300', tz='America/Sao_Paulo')], [60, 'Expedição de documento', Timestamp('2023-08-18 07:20:59-0300', tz='America/Sao_Paulo')], [60, 'Expedição de documento', Timestamp('2023-09-11 05:14:52-0300', tz='America/Sao_Paulo')], [26, 'Distribuição', Timestamp('2023-08-04 08:15:48-0300', tz='America/Sao_Paulo')], [85, 'Petição', Timestamp('2023-08-24 12:53:50-0300', tz='America/Sao_Paulo')], [11010, 'Mero expediente', Timestamp('2023-08-07 15:33:59-0300', tz='America/Sao_Paulo')]]
[[51, 'Conclusão', Timestamp('2023-01-16 06:25:19-0300', tz='America/Sao_Paulo')], [85, 'Petição', Timestamp('2023-07

In [9]:
lista_movimentacoes_completa = list(filter(None, lista_movimentacoes_completa))

In [10]:
print(type(lista_movimentacoes_completa), len(lista_movimentacoes_completa))
print(type(lista_movimentacoes_completa[0]), len(lista_movimentacoes_completa[2]))
print(lista_movimentacoes_completa[5200])
print(lista_movimentacoes_completa[:10])

<class 'list'> 11237
<class 'list'> 4
[['26', "'Distribuição'", '2024-03-07'], ['51', "'Conclusão'", '2024-03-07'], ['60', "'Expediçãodedocumento'", '2024-03-07'], ['36', "'Redistribuição'", '2024-03-07'], ['51', "'Conclusão'", '2024-03-08'], ['60', "'Expediçãodedocumento'", '2024-03-20'], ['85', "'Petição'", '2024-03-24'], ['51', "'Conclusão'", '2024-03-25']]
[[['26', "'Distribuição'", '2022-03-17'], ['51', "'Conclusão'", '2022-03-17']], [['26', "'Distribuição'", '2022-04-28'], ['51', "'Conclusão'", '2022-04-28'], ['60', "'Expediçãodedocumento'", '2022-05-12']], [['26', "'Distribuição'", '2022-03-10'], ['51', "'Conclusão'", '2022-03-10'], ['60', "'Expediçãodedocumento'", '2022-03-14'], ['85', "'Petição'", '2022-04-04']], [['26', "'Distribuição'", '2022-02-07'], ['51', "'Conclusão'", '2022-02-07'], ['60', "'Expediçãodedocumento'", '2022-02-13'], ['85', "'Petição'", '2022-05-02']], [['26', "'Distribuição'", '2022-05-11'], ['51', "'Conclusão'", '2022-05-11'], ['60', "'Expediçãodedocument

In [11]:
print(lista_movimentacoes_completa[0])
print(df_4jefp['movimentos'][0])
print((len(list(df_4jefp['movimentos'][10].split(','))) / 4))

[['26', "'Distribuição'", '2022-03-17'], ['51', "'Conclusão'", '2022-03-17']]
[[26, 'Distribuição', Timestamp('2022-03-17 21:00:00-0300', tz='America/Sao_Paulo')], [51, 'Conclusão', Timestamp('2022-03-17 21:00:00-0300', tz='America/Sao_Paulo')], [60, 'Expedição de documento', Timestamp('2022-04-26 21:00:00-0300', tz='America/Sao_Paulo')], [36, 'Redistribuição', Timestamp('2022-04-26 21:00:00-0300', tz='America/Sao_Paulo')], [51, 'Conclusão', Timestamp('2022-05-05 21:00:00-0300', tz='America/Sao_Paulo')], [60, 'Expedição de documento', Timestamp('2022-05-19 21:00:00-0300', tz='America/Sao_Paulo')], [941, 'Incompetência', Timestamp('2022-03-21 21:00:00-0300', tz='America/Sao_Paulo')], [12099, 'Por decisão do Presidente do STJ - SIRDR', Timestamp('2022-05-18 21:00:00-0300', tz='America/Sao_Paulo')]]
13.0


## Estatísticas

In [12]:
media_movimentacoes = 0.0
stdDev_movimentacoes = 0.0
soma_movimentacoes = 0.0
duracao_media_movimentacoes = 0.0
numero_movimentacoes = 0
soma_duracao_movimentacoes = datetime
dicionario_distribuicao_numero_mov = dict()

for i in lista_movimentacoes_completa:
    print(i)
    if str(len(i)) not in dicionario_distribuicao_numero_mov:
        dicionario_distribuicao_numero_mov[str(len(i))] = []
    dicionario_distribuicao_numero_mov[str(len(i))].append(i)

for i in range(0,len(lista_movimentacoes_completa),1):   
    soma_movimentacoes += len(lista_movimentacoes_completa[i])
    #print(soma_movimentacoes, '\t', i)

for i in range(0,len(lista_movimentacoes_completa),1):
    for j in range(0,(len(lista_movimentacoes_completa[i])-1),4):
        if len(lista_movimentacoes_completa[i][j]) > 4:          
            print(lista_movimentacoes_completa[i][j+4][2] - lista_movimentacoes_completa[i][j][2])
        #print(lista_movimentacoes_completa[i][j])
    print('')

media_movimentacoes = soma_movimentacoes/len(lista_movimentacoes_completa)

print(f'Média das movimentações: {media_movimentacoes}')
    


[['26', "'Distribuição'", '2022-03-17'], ['51', "'Conclusão'", '2022-03-17']]
[['26', "'Distribuição'", '2022-04-28'], ['51', "'Conclusão'", '2022-04-28'], ['60', "'Expediçãodedocumento'", '2022-05-12']]
[['26', "'Distribuição'", '2022-03-10'], ['51', "'Conclusão'", '2022-03-10'], ['60', "'Expediçãodedocumento'", '2022-03-14'], ['85', "'Petição'", '2022-04-04']]
[['26', "'Distribuição'", '2022-02-07'], ['51', "'Conclusão'", '2022-02-07'], ['60', "'Expediçãodedocumento'", '2022-02-13'], ['85', "'Petição'", '2022-05-02']]
[['26', "'Distribuição'", '2022-05-11'], ['51', "'Conclusão'", '2022-05-11'], ['60', "'Expediçãodedocumento'", '2022-05-18'], ['60', "'Expediçãodedocumento'", '2022-05-18']]
[['26', "'Distribuição'", '2022-01-30'], ['51', "'Conclusão'", '2022-01-30'], ['60', "'Expediçãodedocumento'", '2022-02-03'], ['60', "'Expediçãodedocumento'", '2022-02-03'], ['85', "'Petição'", '2022-03-03'], ['85', "'Petição'", '2022-03-15']]
[['26', "'Distribuição'", '2022-05-01'], ['51', "'Conclu

In [13]:
df_teste = pd.DataFrame(lista_movimentacoes_completa)

print(df_teste.head(2))

                                 0                              1   \
0  [26, 'Distribuição', 2022-03-17]  [51, 'Conclusão', 2022-03-17]   
1  [26, 'Distribuição', 2022-04-28]  [51, 'Conclusão', 2022-04-28]   

                                         2     3     4     5     6     7   \
0                                      None  None  None  None  None  None   
1  [60, 'Expediçãodedocumento', 2022-05-12]  None  None  None  None  None   

     8     9     10    11    12    13    14    15    16    17    18    19  \
0  None  None  None  None  None  None  None  None  None  None  None  None   
1  None  None  None  None  None  None  None  None  None  None  None  None   

     20    21    22    23  
0  None  None  None  None  
1  None  None  None  None  


In [14]:
movimentacoes_gabinete = pd.read_csv('dados/movimentos_gabinete.csv', sep=';', header='infer')
movimentacoes_gabinete['codigo'] = movimentacoes_gabinete['vazio.1']
movimentacoes_gabinete.drop(columns=['complemento','vazio', 'vazio.2', 'vazio.1'], inplace=True)
lista_movimentacoes_sentenca = movimentacoes_gabinete[movimentacoes_gabinete['nome'] == 'Julgamento']['codigo'].to_list()
print(movimentacoes_gabinete.head(5))

     quem_faz                   nome  \
0  Magistrado  Cooperação Judiciária   
1  Magistrado                Decisão   
2  Magistrado                Decisão   
3  Magistrado                Decisão   
4  Magistrado                Decisão   

                                  detalhamento  codigo  
0                                          NaN   15185  
1                                          NaN       3  
2        Acolhimento de Embargos de Declaração   15162  
3                       Acolhimento de exceção     133  
4  Acolhimento de exceção de pré-executividade     335  


In [15]:
print(movimentacoes_gabinete[movimentacoes_gabinete['nome'] == 'Julgamento'].head(5))



       quem_faz        nome  \
149  Magistrado  Julgamento   
150  Magistrado  Julgamento   
151  Magistrado  Julgamento   
152  Magistrado  Julgamento   
153  Magistrado  Julgamento   

                                                                detalhamento  \
149                                                                      NaN   
150                                                  Com Resolução do Mérito   
151           Com Resolução do Mérito Acolhimento de Embargos de Declaração    
152  Com Resolução do Mérito Acolhimento em parte de Embargos de Declaração    
153                                        Com Resolução do Mérito Consulta    

     codigo  
149     193  
150     385  
151     198  
152     871  
153   12326  


In [18]:
movimentacoes_secretaria = pd.read_csv('dados/movimentos_secretaria.csv', sep=';', header='infer')
print(movimentacoes_secretaria.head(5))
movimentacoes_secretaria.drop(columns=['Unnamed: 3','complemento'], inplace=True)
lista_movimentacoes_secretaria = movimentacoes_secretaria['codigo'].to_list()
print(movimentacoes_secretaria.head(5))

       quem_faz                                                nome  \
0  Serventuário  Escrivão/Diretor de Secretaria/Secretário Jurídico   
1  Serventuário  Escrivão/Diretor de Secretaria/Secretário Jurídico   
2  Serventuário  Escrivão/Diretor de Secretaria/Secretário Jurídico   
3  Serventuário  Escrivão/Diretor de Secretaria/Secretário Jurídico   
4  Serventuário  Escrivão/Diretor de Secretaria/Secretário Jurídico   

              detalhamento  Unnamed: 3 complemento  codigo  
0      Ações de secretário         NaN         NaN      48  
1              Apensamento         NaN         NaN     135  
2             Arquivamento         NaN         NaN     861  
3  Arquivamento Definitivo  Definitivo         NaN     246  
4  Arquivamento Provisório  Provisório         NaN     245  
       quem_faz                                                nome  \
0  Serventuário  Escrivão/Diretor de Secretaria/Secretário Jurídico   
1  Serventuário  Escrivão/Diretor de Secretaria/Secretário Jurídi

In [36]:
## Esse método não está funcionando a contento. É como se não tivesse nada de julgamento na base. Estou baixando uma nova base (com mais dados) para testar mais e melhor

def calcular_data_sentenca(movimentacoes):
    lista_data_sentenca = list()
    for i in range(0,(len(movimentacoes) -1),1):
        if len(movimentacoes[i]) > 3:
            for j in range(0,(len(movimentacoes[i])-1),3):
                #print(movimentacoes[i][j][0])
                if int(movimentacoes[i][j][0]) in lista_movimentacoes_sentenca:
                    lista_data_sentenca.append(movimentacoes[i][j][2])
                    print(movimentacoes[i][j])
                    

        else: 
            lista_data_sentenca.append(' ')
    return lista_data_sentenca

In [77]:
# funciona ok.
def calcular_tempo_entre_movimentacoes1e2(movimentacoes):
    lista_duracao = list()
    for i in range(0,(len(movimentacoes) -1),1):
        if len(movimentacoes[i]) > 3:
            #print(type(movimentacoes[i][0][2]), '\t', movimentacoes[i][0][2])
            #print(type(movimentacoes[i+1][0][2]), '\t', movimentacoes[i+1][0][2])
            duracao = abs(pd.to_datetime(movimentacoes[i][1][2]) - pd.to_datetime(movimentacoes[i][0][2]))/np.timedelta64(1, 'D')
            lista_duracao.append(duracao)
            #if duracao > 60:
                #print(duracao, '\t', movimentacoes[i][1][2], ' - ', movimentacoes[i][0][2], '\t', movimentacoes[i])                  

        else: 
            lista_duracao.append(' ')
            
    return lista_duracao        

In [78]:
calcular_data_sentenca(lista_movimentacoes_completa)
print(len(calcular_tempo_entre_movimentacoes1e2(lista_movimentacoes_completa)))

11236
